In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyodbc

In [65]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, PWD = password)
        print("MS SQL Server Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return conn

In [66]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'VAIO'
database = 'master'
user = 'sa'
password = '14531453'
conn = create_server_connection(driver, server, database, user, password)

MS SQL Server Database connection successful


In [67]:
conn.autocommit = True

In [5]:
def create_database(conn, create_database_query):
    csr = conn.cursor()
    try:
        csr.execute(create_database_query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [13]:
create_database_query = 'CREATE DATABASE Test_DB'
create_database(conn, create_database_query)

Database created successfully


In [6]:
def execute_query(conn, query):
    csr = conn.cursor()
    try:
        csr.execute(query)
        conn.commit()
        print("Query successful")
    
    except Error as err:
        print(f"Error: '{err}'")

In [68]:
query ='USE Test_DB'

In [8]:
execute_query(conn, query)

Query successful


In [46]:
query = 'CREATE TABLE TestA ( \
             ID int IDENTITY(1,1) NOT NULL, \
             FirstName varchar(255) NOT NULL, \
             LastName varchar(255) NOT NULL, \
             PRIMARY KEY(ID))'
execute_query(conn, query)

Query successful


In [9]:
csr = conn.cursor()

In [10]:
csr.execute("INSERT TestA (FirstName, LastName) VALUES ('Aaaa', 'Bbbb')")
conn.commit()

In [11]:
csr.execute("INSERT TestA (FirstName, LastName) VALUES ('Bruce', 'Lee')")
conn.commit()

In [12]:
csr.execute("INSERT TestA (FirstName, LastName) VALUES ('Bob', 'Marley')")
conn.commit()

In [13]:
csr.execute("INSERT TestA (FirstName, LastName) VALUES ('Mel', 'Gibson')")
conn.commit()

In [54]:
csr.execute("delete from TestA where FirstName = ?", 'Aaaa')

In [14]:
conn.commit()

In [15]:
print(csr.rowcount, 'row(s) deleted')

1 row(s) deleted


In [16]:
deleted = csr.execute("delete from TestA where FirstName = 'Cccc'").rowcount

In [17]:
conn.commit()

In [18]:
print(deleted, 'row(s) deleted')

0 row(s) deleted


In [19]:
csr = conn.cursor()
csr.execute('SELECT FirstName, LastName FROM TestA')
row = csr.fetchone()
if row:
    print(row)

('Bruce', 'Lee')


In [20]:
row.FirstName

'Bruce'

In [21]:
row.LastName

'Lee'

In [22]:
print("LastName: ", row.LastName)

LastName:  Lee


In [23]:
csr.execute('SELECT FirstName, LastName FROM TestA')
while True:
    row = csr.fetchone()
    if not row:
        break
    print('FirstName:', row.FirstName)

FirstName: Bruce
FirstName: Bob
FirstName: Bob
FirstName: Mel
FirstName: Aaaa
FirstName: Bruce
FirstName: Bob
FirstName: Mel


In [26]:
csr.execute('SELECT ID, FirstName, LastName FROM TestA')
while True:
    row = csr.fetchone()
    if not row:
        break
    print(row.ID,row.FirstName,row.LastName)

4 Bruce Lee
5 Bob Marley
6 Bob Marley
7 Mel Gibson
8 Aaaa Bbbb
9 Bruce Lee
10 Bob Marley
11 Mel Gibson


In [28]:
csr.execute('SELECT ID, FirstName, LastName FROM TestA')
rows = csr.fetchall()
rows

[(4, 'Bruce', 'Lee'),
 (5, 'Bob', 'Marley'),
 (6, 'Bob', 'Marley'),
 (7, 'Mel', 'Gibson'),
 (8, 'Aaaa', 'Bbbb'),
 (9, 'Bruce', 'Lee'),
 (10, 'Bob', 'Marley'),
 (11, 'Mel', 'Gibson')]

In [ ]:
csr.execute("""SELECT ID, FirstName, LastName 
            FROM TestA""")
rows = csr.fetchall()
rows

In [29]:
for row in rows:
    print(row.ID,row.FirstName,row.LastName)

4 Bruce Lee
5 Bob Marley
6 Bob Marley
7 Mel Gibson
8 Aaaa Bbbb
9 Bruce Lee
10 Bob Marley
11 Mel Gibson


In [30]:
type(rows)

list

In [31]:
csr.execute("""SELECT ID, FirstName, LastName 
            FROM TestA
            WHERE FirstName = ? and LastName = ?""",['Mel','Gibson'])
rows = csr.fetchall()
rows

[(7, 'Mel', 'Gibson'), (11, 'Mel', 'Gibson')]

In [32]:
csr.execute("""SELECT ID, FirstName, LastName 
            FROM TestA
            WHERE FirstName = ? and LastName = ?""",['Mel','Gibson'])
rows = csr.fetchmany(4)
rows

[(7, 'Mel', 'Gibson'), (11, 'Mel', 'Gibson')]

In [33]:
csr.execute("""SELECT ID, FirstName, LastName 
            FROM TestA
            WHERE FirstName = ? and LastName = ?""",['Mel','Gibson'])
rows = csr.fetchmany(1)
rows

[(7, 'Mel', 'Gibson')]

In [38]:
csr.execute("""SELECT ID, FirstName, LastName 
            FROM TestA""")
rows = csr.fetchmany(2)
rows

[(4, 'Bruce', 'Lee'), (5, 'Bob', 'Marley')]

In [39]:
csr.execute("""SELECT ID, FirstName, LastName 
            FROM TestA""")
rows = csr.fetchmany(2) # csr.execute sorgusunu yeniden çalıştırdığımız için aynı değer, verir
rows

[(4, 'Bruce', 'Lee'), (5, 'Bob', 'Marley')]

In [48]:
csr.execute("""SELECT ID, FirstName, LastName 
            FROM TestA""")
rows = csr.fetchmany(3)
rows

[(4, 'Bruce', 'Lee'), (5, 'Bob', 'Marley'), (6, 'Bob', 'Marley')]

In [49]:
rows = csr.fetchmany(1) # csr.execute sorgusunu çalıştırmadığımız için cursor kaldığı yerden sonra istenilen kadar verdi.
rows

[(7, 'Mel', 'Gibson')]

In [71]:
def read_query(conn, query):
    csr = conn.cursor()
    results = None
    csr.execute(query)
    results = csr.fetchall()
    csr.close()
    del csr
    return results

In [ ]:
query = """SELECT ID, FirstName, LastName FROM TestA"""
read_query(conn, query)

In [64]:
conn.close()

In [73]:
csr = conn.cursor()